In [1]:
import setuptools
import pandas as pd
import numpy as np
import itertools
from pprint import pprint
import timeit
import functools
import operator
from IPython.display import display
import xlsxwriter
import psutil

In [2]:
#display all of dataframe
pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [3]:
#read winning numbers into dataframes
lotnumbs = "C:\\Users\\surfacepro3\\Desktop\\lottery files\\Lottery_Mega_Millions_Winning_Numbers__Beginning_2002.csv"

lotnumbsdf_full = pd.read_csv(lotnumbs)

lotnumbsdf = lotnumbsdf_full[['Winning Numbers','Mega Ball']]

new = lotnumbsdf["Winning Numbers"].str.split(" ", expand = True)

lotnumbsdf["n1"]= new[0]
lotnumbsdf["n2"]= new[1]
lotnumbsdf["n3"]= new[2]
lotnumbsdf["n4"]= new[3]
lotnumbsdf["n5"]= new[4]

lotnumbsdf[7] = lotnumbsdf['Mega Ball'] 
lotnumbsdf.rename(columns = {7:'MB'}, inplace = True) 

lotnumbsdf.drop(columns =["Winning Numbers", "Mega Ball"], inplace = True)

df = pd.concat([lotnumbsdf_full[['Draw Date']],lotnumbsdf[['n1','n2','n3','n4','n5','MB']]], axis=1)
df.head(2000)
#df.info()

df['Draw Date'] = df['Draw Date'].astype('datetime64[ns]') 
df[['n1','n2','n3','n4','n5','MB']] = df[['n1','n2','n3','n4','n5','MB']].astype(int)

df_whole = df.copy()

df.info()
df.head(5)

#current numbers and rules start from Oct. 28 2017
df_nf = df[(df['Draw Date'] >= '2017-10-28')]

df = df_nf.reset_index(drop=True)

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1847 entries, 0 to 1846
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Draw Date  1847 non-null   datetime64[ns]
 1   n1         1847 non-null   int32         
 2   n2         1847 non-null   int32         
 3   n3         1847 non-null   int32         
 4   n4         1847 non-null   int32         
 5   n5         1847 non-null   int32         
 6   MB         1847 non-null   int32         
dtypes: datetime64[ns](1), int32(6)
memory usage: 57.8 KB


C:\Python3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Draw Date,n1,n2,n3,n4,n5,MB
0,2017-10-31,6,28,31,52,53,12
1,2017-11-03,10,22,42,61,69,3
2,2017-11-07,1,54,60,68,69,11
3,2017-11-10,6,23,38,42,58,24
4,2017-11-14,1,14,21,22,28,19


In [4]:
df_wnumonly = df.copy()
df_megaball = df.copy()
df_allnums = df.copy()
df_studyOCTBCK = df.copy()

df_wnumonly.drop(columns =["Draw Date", "MB"], inplace = True)
df_megaball.drop(columns =["Draw Date","n1","n2","n3","n4","n5"], inplace = True)
df_allnums.drop(columns =["Draw Date"], inplace = True)

In [5]:
#Percentages of megaball number hits
count = pd.Series(df_megaball.squeeze().values.ravel()).value_counts()
megaballsts =pd.DataFrame({'Measure': count.index, 'Count':count.values, 'Percentage':(count/count.sum()).values})
megaballsts.head(2000)

,Measure,Count,Percentage
0,11,15,0.063291
1,9,14,0.059072
2,10,14,0.059072
3,22,13,0.054852
4,20,13,0.054852
5,19,11,0.046414
6,1,10,0.042194
7,23,10,0.042194
8,21,10,0.042194
9,16,10,0.042194


In [6]:
#Percentages of winning numbers hit
count = pd.Series(df_wnumonly.squeeze().values.ravel()).value_counts()
wnstats =pd.DataFrame({'Measure': count.index, 'Count':count.values, 'Percentage':(count/count.sum()).values})
wnstats.head(2500)

,Measure,Count,Percentage
0,62,26,0.021941
1,31,26,0.021941
2,48,24,0.020253
3,10,24,0.020253
4,14,24,0.020253
5,17,24,0.020253
6,4,23,0.019409
7,42,23,0.019409
8,11,22,0.018565
9,24,22,0.018565


In [7]:
wnarr = wnstats[['Measure']].to_numpy()
wnpercarr = wnstats[['Percentage']].to_numpy()

megaballstsarr = megaballsts[['Measure']].to_numpy()
megaballstsperarr = megaballsts[['Percentage']].to_numpy()

megaballstsperarr = megaballstsperarr.ravel()
megaballstsarr = megaballstsarr.ravel()

wnarr = wnarr.ravel()
wnpercarr = wnpercarr.ravel()

In [8]:
print (megaballstsarr)
print (megaballstsperarr)

[11  9 10 22 20 19  1 23 21 16 24  4  5  7 13 14 12  6  3  2 17 25 18  8
 15]
[0.06329114 0.05907173 0.05907173 0.05485232 0.05485232 0.0464135
 0.04219409 0.04219409 0.04219409 0.04219409 0.04219409 0.03797468
 0.03797468 0.03797468 0.03797468 0.03797468 0.03375527 0.03375527
 0.03375527 0.03375527 0.02953586 0.02953586 0.02531646 0.02109705
 0.02109705]


In [9]:
#this will make random choices based off the percentages giving in the winning numbers
#np.random.choice(5, 3, replace=False, p=[0.1, 0, 0.3, 0.6, 0])
#np.random.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])
wnumbsr = np.random.choice(wnarr, 5, replace=False, p= wnpercarr)
print (wnumbsr)

mnumbsr = np.random.choice(megaballstsarr, 1, replace=False, p= megaballstsperarr)
print (mnumbsr)

[47 53 23 54 39]
[8]


In [10]:
df_whole.drop(columns =["Draw Date"], inplace = True)
df_wholearr = df_whole.to_numpy()
df_wholearr = df_wholearr.astype(int)
print(df_wholearr)

[[15 18 25 33 47 30]
 [ 4 28 39 41 44  9]
 [ 2  4 32 44 52 36]
 ...
 [17 36 47 51 62 21]
 [28 31 33 57 62 19]
 [32 48 50 51 64 10]]


In [11]:
numbersp = []
x = 0
while x < 6:
 wnp = np.random.choice(wnarr, 5, replace=False, p= wnpercarr)
 mnp = np.random.choice(megaballstsarr, 1, replace=False, p= megaballstsperarr)
 
 #print (wn)
 #print (mg)
 
 wnp = np.sort(wnp)
 
 #print (wn)
 
 cnp = np.append(wnp, mnp)
 print(cnp)
 #cnp = np.array(cnp)}|
 if  (not(np.any([(cnp == x).all() for x in df_wholearr]))) and (11.6 <= np.mean(cnp) <= 48.16) and (70 <= np.sum(cnp) <= 289):
    print(cnp)
    numbersp.append(cnp)
    #print (cn)
    x += 1

    

#np.any([(cnp == x).all() for x in df_wholearr])) and



[ 1  3 37 39 64  8]
[ 1  3 37 39 64  8]
[ 5 19 35 38 44 20]
[ 5 19 35 38 44 20]
[13 14 24 31 44  3]
[13 14 24 31 44  3]
[ 5 28 29 31 70 21]
[ 5 28 29 31 70 21]
[29 33 40 43 67 16]
[29 33 40 43 67 16]
[ 3  8 12 20 70 23]
[ 3  8 12 20 70 23]


In [12]:
print(numbersp)

[array([ 1,  3, 37, 39, 64,  8], dtype=int64), array([ 5, 19, 35, 38, 44, 20], dtype=int64), array([13, 14, 24, 31, 44,  3], dtype=int64), array([ 5, 28, 29, 31, 70, 21], dtype=int64), array([29, 33, 40, 43, 67, 16], dtype=int64), array([ 3,  8, 12, 20, 70, 23], dtype=int64)]
